#### Here BeautifulSoup is used to download the Ontario Electrical demand data from http://reports.ieso.ca/public/DemandZonal/

#### The temperature, dewpoint, wind related hourly data was downloaded from https://toronto.weatherstats.ca/download.html

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib import request

In [2]:
import pandas as pd
import numpy as np

In [3]:
r = requests.Session().get('http://reports.ieso.ca/public/DemandZonal/')
soup = BeautifulSoup(r.content, 'html.parser')

We only want the recent data from 2017. Using regular expression to extract the links for 2017 or greater.

In [4]:
years_list = []
[years_list.extend(list(set(re.findall(r'20[0-2][0-9]',str(soup.select("a[href$='.csv']")[item]))))) for item in range(len(soup.select("a[href$='.csv']")))]
unique_list = list(set(years_list))
unique_list.sort()
unique_list = [item for item in unique_list if int(item)>=2017]

Since, multiple versions exist, we choose the latest version for a given year.

In [5]:
index = [max(loc for loc, val in enumerate(years_list) if val == item)+1 for item in unique_list]

In [6]:
relevant_files = [soup.select("a[href$='.csv']")[item] for item in index]
display(relevant_files)

[<a href="PUB_DemandZonal_2017_v1.csv">PUB_DemandZonal_2017_v1.csv</a>,
 <a href="PUB_DemandZonal_2018_v254.csv">PUB_DemandZonal_2018_v254.csv</a>,
 <a href="PUB_DemandZonal_2019_v395.csv">PUB_DemandZonal_2019_v395.csv</a>,
 <a href="PUB_DemandZonal_2020_v382.csv">PUB_DemandZonal_2020_v382.csv</a>,
 <a href="PUB_DemandZonal_2021_v18.csv">PUB_DemandZonal_2021_v18.csv</a>]

Writing each text into a CSV file in the Data folder

In [7]:
for file in range(len(relevant_files)):
    
    response = request.urlopen("http://reports.ieso.ca/public/DemandZonal/" + relevant_files[file]['href'])
    csv = response.read()

    csvstr = str(csv).strip("b'")
    lines = csvstr.split("\\n")
    f = open("Data\\Demand_" + unique_list[file] + ".csv", "w")
    for line in lines:
       f.write(line + "\n")
    f.close()

#### Organizing the data

Importing the CSV files for demand from 2017 to 2021 and the weather data

In [8]:
data1 = pd.read_csv('Data/Demand_2017.csv',header=3)
data2 = pd.read_csv('Data/Demand_2018.csv',header=3)
data3 = pd.read_csv('Data/Demand_2019.csv',header=3)
data4 = pd.read_csv('Data/Demand_2020.csv',header=3)
data5 = pd.read_csv('Data/Demand_2021.csv',header=3)
data = pd.read_csv('Data/Toronto_weather.csv')
data = data.loc[::-1].reset_index(drop=True)

The weather data starts from 2021 and goes until 2017, while the demand data is the other way around. Therefore, we invert the weather data.

In [9]:
data['demand'] = pd.concat([data1['Toronto'],data2['Toronto'],data3['Toronto'],data4['Toronto'],data5['Toronto']],axis=0).reset_index(drop=True)

In [10]:
data.to_csv('Data/Toronto_data.csv',index=False)